In [23]:
%load_ext autoreload
%autoreload 2

import os
curpath = os.getcwd()
os.chdir(curpath.split("core")[0])

In [28]:
import openai
import json
import time
from typing import List, Dict, Any, Tuple, Union
import urllib3
from xml.dom.minidom import parseString
from core.services.provider import creator, creator_defaults

# from prompts import SIMPLE_TS_PROMPTS, SIMPLE_TS_WITH_REF_PROMPTS
# from core.services.topic_segregation.utils import add_dicts, calculate_cost_gpt4_8k

from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
creator(**creator_defaults, messages=[{'role': 'user', 'content': 'Hello, world!'}])

In [27]:
def get_google_news_results(term:str, count: int):
    http = urllib3.PoolManager()
    results = []
    response = http.request('GET', 'http://news.google.com/news?q=%s&lang=en&output=rss' % term)
    obj = parseString(response.data.decode('utf-8'))
    items = obj.getElementsByTagName('item')  # Get each item
    for item in items[:count]:
        title, link = '', ''
        for node in item.childNodes:
            if node.nodeName == 'title':
                title = node.childNodes[0].data
            elif node.nodeName == 'link':
                link = node.childNodes[0].data
        results.append((title, link))
    return results

In [31]:
results = get_google_news_results('Rajasthan%20latest%20news', 100)
titles = []
links = []
for title, link in results:
    titles.append(title)
    links.append(link)

In [24]:
links[0]

'https://news.google.com/rss/articles/CBMiSmh0dHBzOi8vdGhld2lyZS5pbi9wb2xpdGljcy9yYWphc3RoYW4tY29uZ3Jlc3MtbWFkaHlhLXByYWRlc2gtY2hoYXR0aXNnYXJo0gFYaHR0cHM6Ly9tLnRoZXdpcmUuaW4vYXJ0aWNsZS9wb2xpdGljcy9yYWphc3RoYW4tY29uZ3Jlc3MtbWFkaHlhLXByYWRlc2gtY2hoYXR0aXNnYXJoL2FtcA?oc=5'

In [25]:
titles[0]

'Win or Lose: Rajasthan Shows The Way Forward for the Congress - The Wire'

In [32]:
title_txt = "\n".join(titles)

with open ('news_data_raj_en.txt', "wb") as f:
    f.write(title_txt.encode('utf-8'))

In [4]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time

class InformationScraper:
    def __init__(self, queries, frequency='daily'):
        self.queries = queries
        self.frequency = frequency
        self.results = []

    def fetch_news(self, query):
        # Replace this URL with the actual news site or API you're planning to scrape
        url = f'https://news.google.com/search?q={query}'
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to retrieve news for query: {query}")
            return None

    def parse_news(self, html_content):
        soup = BeautifulSoup(html_content, 'html.parser')
        # Modify the selectors based on the actual structure of the news site
        articles = soup.find_all('article')
        news_items = []
        for article in articles:
            title = article.find('h3').get_text() if article.find('h3') else 'No Title'
            link = article.find('a', href=True)['href'] if article.find('a', href=True) else 'No Link'
            news_items.append({'title': title, 'link': link})
        return news_items

    def scrape(self):
        for query in self.queries:
            html_content = self.fetch_news(query)
            if html_content:
                news_items = self.parse_news(html_content)
                self.results.extend(news_items)

    def run(self):
        if self.frequency == 'daily':
            while True:
                print(f"Starting scrape at {datetime.now()}")
                self.scrape()
                print(f"Scrape complete at {datetime.now()}")
                break
                time.sleep(86400)  # Sleep for 1 day
        elif self.frequency == 'hourly':
            while True:
                print(f"Starting scrape at {datetime.now()}")
                self.scrape()
                print(f"Scrape complete at {datetime.now()}")
                break
                time.sleep(3600)  # Sleep for 1 hour
        # Add more frequency options as needed

    def get_results(self):
        return self.results

# Example usage:
queries = ['education', 'e-learning', 'online courses']
scraper = InformationScraper(queries, frequency='daily')
scraper.run()


Starting scrape at 2023-11-10 13:51:05.010343
Scrape complete at 2023-11-10 13:51:10.337551


In [ ]:
https://news.google.com/articles/CBMihgFodHRwczovL3d3dy5idXNpbmVzcy1zdGFuZGFyZC5jb20vZWNvbm9teS9uZXdzL3RheC13aW5kZmFsbC1naXZlcy1tb2RpLWdvdnQtc2NvcGUtdG8tc3BlbmQtbW9yZS1vbi13ZWxmYXJlLXNjaGVtZXMtMTIzMTEwODAwNTEzXzEuaHRtbNIBigFodHRwczovL3d3dy5idXNpbmVzcy1zdGFuZGFyZC5jb20vYW1wL2Vjb25vbXkvbmV3cy90YXgtd2luZGZhbGwtZ2l2ZXMtbW9kaS1nb3Z0LXNjb3BlLXRvLXNwZW5kLW1vcmUtb24td2VsZmFyZS1zY2hlbWVzLTEyMzExMDgwMDUxM18xLmh0bWw?hl=en-IN&gl=IN&ceid=IN%3Aen

In [5]:
links = []
gnews = "https://news.google.com"

for res in scraper.results:
    links.append(gnews + res['link'][1:])


In [7]:
# len(links)

292

In [6]:
import requests
from bs4 import BeautifulSoup, TemplateString
from tqdm import tqdm
import time

def scraper(urls):
    
    results = []
    for url in tqdm(urls):

        # Send a GET request to the URL
        try:
            response = requests.get(url)
            time.sleep(0.5)
        except:
            print(f'Failed to get {url}')
            continue

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the page using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
        
            for data in soup(['head', 'header', 'style', 'script', 'g', 'polygon', 'path', 'svg', 'option', 'footer']):
                # Remove tags
                data.decompose()

            article_text = ' '.join(paragraph.text.strip() for paragraph in soup.find_all('p'))
            
            # Extract the headline
            headline = soup.find('h1')
            if headline:
                headline = headline.text.strip()
            else:
                headline = ''

            result = headline + '\n' + article_text
            results.append(result)

        else:
            print("Failed to retrieve the webpage. Status code:", response.status_code)
            print(f"Error in URL: {url}")

    return results


In [7]:
results = scraper(links[:50])

  0%|          | 0/50 [00:00<?, ?it/s]

 16%|█▌        | 8/50 [00:14<01:16,  1.81s/it]

Failed to retrieve the webpage. Status code: 401
Error in URL: https://news.google.com/articles/CBMigwFodHRwczovL3d3dy5oaW5kdXN0YW50aW1lcy5jb20vbGlmZXN0eWxlL2FydC1jdWx0dXJlL25hdGlvbmFsLWVkdWNhdGlvbi1kYXktMjAyMy1kYXRlLWhpc3RvcnktYW5kLXNpZ25pZmljYW5jZS0xMDE2OTk1MjA5MzMyMzAuaHRtbNIBhwFodHRwczovL3d3dy5oaW5kdXN0YW50aW1lcy5jb20vbGlmZXN0eWxlL2FydC1jdWx0dXJlL25hdGlvbmFsLWVkdWNhdGlvbi1kYXktMjAyMy1kYXRlLWhpc3RvcnktYW5kLXNpZ25pZmljYW5jZS0xMDE2OTk1MjA5MzMyMzAtYW1wLmh0bWw?hl=en-IN&gl=IN&ceid=IN%3Aen


 38%|███▊      | 19/50 [00:33<00:50,  1.64s/it]

Failed to retrieve the webpage. Status code: 401
Error in URL: https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5oaW5kdXN0YW50aW1lcy5jb20vZWR1Y2F0aW9uL25ld3MvZHBzcnUtaW5rcy1tb3Utd2l0aC1ha3Vtcy1kcnVncy10by1lbmhhbmNlLXBoYXJtYS1lZHVjYXRpb24tYW5kLXJlc2VhcmNoLTEwMTY5OTAxMTY5NDU5Mi5odG1s0gGPAWh0dHBzOi8vd3d3LmhpbmR1c3RhbnRpbWVzLmNvbS9lZHVjYXRpb24vbmV3cy9kcHNydS1pbmtzLW1vdS13aXRoLWFrdW1zLWRydWdzLXRvLWVuaGFuY2UtcGhhcm1hLWVkdWNhdGlvbi1hbmQtcmVzZWFyY2gtMTAxNjk5MDExNjk0NTkyLWFtcC5odG1s?hl=en-IN&gl=IN&ceid=IN%3Aen


...

Failed to retrieve the webpage. Status code: 403
Error in URL: https://news.google.com/articles/CBMiLmh0dHBzOi8vc3RhdGVtZW50cy5xbGQuZ292LmF1L3N0YXRlbWVudHMvOTkxMjnSAQA?hl=en-IN&gl=IN&ceid=IN%3Aen


 82%|████████▏ | 41/50 [01:44<01:13,  8.17s/it]

Failed to get https://news.google.com/articles/CBMiZ2h0dHBzOi8vd3d3Lm1vbnRldmFsbG8uZWR1L3VtLXRvLWhvc3QtNTFzdC1hbm51YWwtYWxhYmFtYS1taXNzaXNzaXBwaS1zb2NpYWwtd29yay1lZHVjYXRpb24tY29uZmVyZW5jZS_SAQA?hl=en-IN&gl=IN&ceid=IN%3Aen


100%|██████████| 50/50 [02:11<00:00,  2.62s/it]

Failed to retrieve the webpage. Status code: 403
Error in URL: https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnRoZWxhbmNldC5jb20vam91cm5hbHMvbGFuY2V0L2FydGljbGUvUElJUzAxNDAtNjczNigyMykwMjQ2Mi01L2Z1bGx0ZXh00gEA?hl=en-IN&gl=IN&ceid=IN%3Aen


In [10]:
len(results)

with open("test_news.txt", 'wb') as f:
    for result in results:
        f.write(result.encode('utf-8').strip())

In [36]:
import requests
from bs4 import BeautifulSoup, TemplateString

# def scraper(url):
    # The URL of the article to scrape
url = "https://me.mashable.com/culture/34493/northern-lights-paint-bulgarias-sky-in-eerie-red-hue-see-pics"
url = "https://www.ndtv.com/india-news/cbi-inquiry-against-mahua-moitra-ordered-by-anti-corruption-panel-bjp-mp-4557198"
url = "https://thewire.in/security/india-armed-forces-politicisation-by-default-or-design"
url = "https://www.washingtonpost.com/politics/2023/11/07/abortion-ohio-kentucky-virginia-elecitons/"
url = "https://www.theinformation.com/articles/american-ai-startups-raise-money-from-top-chinese-vc-firms-including-sequoia-capital-china"
url = "https://www.nytimes.com/2023/11/08/business/energy-environment/electric-vehicles-biden.html"

    # Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content of the page using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # soup = BeautifulSoup(html, "html.parser")
 
    for data in soup(['head', 'header', 'style', 'script', 'g', 'polygon', 'path', 'svg', 'option', 'footer']):
        # Remove tags
        data.decompose()
    # temp = TemplateString(response.content)
    print(soup)
    print()

    article_text = ' '.join(paragraph.text.strip() for paragraph in soup.find_all('p'))

    # print(article_text)
    # print(len(soup))
    # print(temp)
    # print(soup)

    # soup.find_all('script')
    # soup.find_all()
    
    # Extract the headline
    headline = soup.find('h1')
    if headline:
        headline = headline.text.strip()
    
#     # Try to find the container of the article content
#     # The class or tag here is hypothetical and should be replaced with the actual class or tag that contains the article text.
#     article_content = soup.find('article') 
#     # or soup.find('main')  # Adjust the class or tag as needed
    
#     # Check if the article content was found
    # if article_content:
    #     paragraphs = article_content.find_all('p')
    #     article_text = ' '.join(paragraph.text.strip() for paragraph in paragraphs)
    # else:
    #     article_text = "Could not find the article content."
    
    # Print the extracted information
    print("Headline:", headline)
    print("Article Text:", article_text)
# else:
#     print("Failed to retrieve the webpage. Status code:", response.status_code)

        
        


In [10]:
url = "https://www.ndtv.com/india-news/cbi-inquiry-against-mahua-moitra-ordered-by-anti-corruption-panel-bjp-mp-4557198"

scraper(url)

Headline: CBI Probe Against Mahua Moitra Ordered By Anti-Graft Panel, Claims BJP MP
Article Text: Anti-corruption body Lokpal has ordered a probe by the Central Bureau of Investigation into the corruption allegations against Trinamool Congress MP Mahua Moitra, the BJP's Nishikant Dubey posted today on X, formerly Twitter. Ms Moitra has been embroiled in cash for query allegations levelled by Mr Dubey, which is under the lens of the Parliamentary Ethics Committee. In a letter to Lok Sabha Speaker Om Birla, he had demanded her immediate suspension from parliament. "On basis of my complaint, Lokpal has ordered a CBI probe into the Mahua Moitra's corruption that compromises national security," read a rough translation of BJP MP's post. Nishikant Dubey has alleged that Ms Moitra accepted cash from businessman Darshan Hiranandani to ask questions in parliament on his behalf to target Prime Minister Narendra Modi and business rival Adani Group.  He has also claimed that Ms Moitra, by sharing 